<h2><b>IA</b></h2>

In [134]:
import tkinter as tk
from random import randint
from PIL import Image, ImageTk
import time


MOVE_INCREMENT = 20
MOVES_PER_SECOND = 15
GAME_SPEED = 50 // MOVES_PER_SECOND
GAME_HEIGHT = 600
GAME_WIDTH = 600

class Snake(tk.Canvas):
    def __init__(self):
        super().__init__(
            width=GAME_WIDTH, height=GAME_HEIGHT, background="black", highlightthickness=0
        )

        self.snake_positions = [(100, 40), (80, 40), (60, 40)]
        self.food_position = self.set_new_food_position()
        self.direction = "Right"

        self.score = 0
        self.hamiltonian_path = []
        
        self.load_assets()
        self.create_objects()

        self.bind_all("<Key>", self.on_key_press)

        self.after(GAME_SPEED, self.perform_actions)
        
        self.index = 0

    def load_assets(self):
        try:
            self.snake_body_image = Image.open("assets/snake.png")
            self.snake_body = ImageTk.PhotoImage(self.snake_body_image)

            self.food_image = Image.open("assets/food.png")
            self.food = ImageTk.PhotoImage(self.food_image)
        except IOError as error:
            root.destroy()
            raise

    def create_objects(self):
        self.create_text(
            35, 12, text=f"Score: {self.score}", tag="score", fill="#fff", font=10
        )

        for x_position, y_position in self.snake_positions:
            self.create_image(
                x_position, y_position, image=self.snake_body, tag="snake"
            )

        self.create_image(*self.food_position, image=self.food, tag="food")
        self.create_rectangle(7, 27, 593, 593, outline="#525d69")

    def check_collisions(self):
        self.head_x_position, self.head_y_position = self.snake_positions[0]

        return (
            self.head_x_position in (0, GAME_WIDTH)
            or self.head_y_position in (20, GAME_HEIGHT)
            or (self.head_x_position, self.head_y_position) in self.snake_positions[1:]
        )

    def check_food_collision(self):
        if self.snake_positions[0] == self.food_position:
            self.score += 1
            
            self.snake_positions.append(self.snake_positions[-1])
            self.create_image(
                *self.snake_positions[-1], image=self.snake_body, tag="snake"
            )
            
            self.food_position = self.set_new_food_position()
            self.coords(self.find_withtag("food"), *self.food_position)

            score = self.find_withtag("score")
            self.itemconfigure(score, text=f"Score: {self.score}", tag="score")
            
    def end_game(self):
        self.delete(tk.ALL)
        self.create_text(
            self.winfo_width() / 2,
            self.winfo_height() / 2,
            text=f"Game over! You scored {self.score}!",
            fill="#fff",
            font=14
        )

    def move_snake(self):
        head_x_position, head_y_position = self.snake_positions[0]

        if self.direction == "Left":
            new_head_position = (head_x_position - MOVE_INCREMENT, head_y_position)
        elif self.direction == "Right":
            new_head_position = (head_x_position + MOVE_INCREMENT, head_y_position)
        elif self.direction == "Down":
            new_head_position = (head_x_position, head_y_position + MOVE_INCREMENT)
        elif self.direction == "Up":
            new_head_position = (head_x_position, head_y_position - MOVE_INCREMENT)

        self.snake_positions = [new_head_position] + self.snake_positions[:-1]

        for segment, position in zip(self.find_withtag("snake"), self.snake_positions):
            self.coords(segment, position)

    def on_key_press(self, e):
        new_direction = e.keysym

        all_directions = ("Up", "Down", "Left", "Right")
        opposites = ({"Up", "Down"}, {"Left", "Right"})

        if (
            new_direction in all_directions
            and {new_direction, self.direction} not in opposites
        ):
            self.direction = new_direction
            
    def shortest_path(self):
        head_x_position, head_y_position = self.snake_positions[0]
        food_x_position, food_y_position = self.food_position
        distance_to_food_x = head_x_position - food_x_position
        distance_to_food_y = head_y_position - food_y_position
        while distance_to_food_y != 0:
            if distance_to_food_y > 0:
                self.direction = "Up"
                distance_to_food_y -= 20
            else:
                self.direction = "Down"
                distance_to_food_y += 20
        while distance_to_food_x != 0:
            if distance_to_food_x > 0:
                self.direction = "Left"
                distance_to_food_x -= 20
            else:
                self.direction = "Right"
                distance_to_food_x += 20

    def perform_actions(self):
        if self.check_collisions():
            self.end_game()
            return
        
        self.ia_hamiltonian()
        self.check_food_collision()
        self.move_snake()
        self.after(GAME_SPEED, self.perform_actions)

    def set_new_food_position(self):
        while True:
            x_position = randint(1, 29) * MOVE_INCREMENT
            y_position = randint(3, 29) * MOVE_INCREMENT
            food_position = (x_position, y_position)

            if food_position not in self.snake_positions:
                return food_position
            
    def ia(self):
        self.shortest_path()
        self.disable_keyboard()
        
    def ia_hamiltonian(self):
        self.hamiltonian()
        print(self.head_y_position)
        
    
    def disable_keyboard(self):
        self.bind_all("<Key>", "break")
        
    def hamiltonian(self):
        self.hamiltonian_path = []
        if self.head_y_position == GAME_WIDTH - 20:
            if self.head_x_position - 20 == 0:
                self.direction = "Up"
            else:
                self.direction = "Left"
        elif self.direction == "Right":
            if self.head_x_position + 20 != GAME_WIDTH:
                self.direction = "Right"
                pass
            else:
                self.direction = "Down"
        elif self.direction == "Down":
            if self.head_x_position + 20 == GAME_WIDTH:
                self.direction = "Left"
                pass
            else:
                self.direction = "Right"
        elif self.direction == "Left":
            if self.head_x_position - 20 != 20:
                self.direction = "Left"
            else:
                self.direction = "Down"
        elif self.direction == "Up" and self.head_y_position - 20 == 20:
            self.direction = "Right"
        
root = tk.Tk()
root.title("Snake")
root.resizable(False, False)
root.tk.call("tk", "scaling", 4.0)

board = Snake()
board.pack()

root.mainloop()

40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
40
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
100
100
100
100
100
100
100
100
